In [2]:
import multiprocessing
import dadi
import pandas as pd
import numpy as np
import random
import argparse
from itertools import product
import matplotlib.pyplot as plt
import tskit
import numpy
import seaborn as sns

In [4]:
ns = np.array([20])
pts_l = [20, 40, 60]

<h2>Stationary population</h2>

In [4]:
fs = dadi.Spectrum.from_file(inFile)

params = ["notused"]
func = dadi.Demographics1D.snm
# Make the extrapolating version of our demographic model function.
func_ex = dadi.Numerics.make_extrap_func(func)
model = func_ex( params, ns, pts_l)

# Likelihood of the data given the model AFS.
ll_model = dadi.Inference.ll_multinom(model, fs)
theta = dadi.Inference.optimal_sfs_scaling(model, fs)
N = theta/(4*0.81e-8*213936105)
with open(outFile, 'a') as f:
    f.write(str(theta) + '\t' + str(N) + '\t' + str(N/10000) + "\t" + str(ll_model) + '\n')

<h2>Population size change inference</h2>

In [ ]:
#Add input and output file details
fs = dadi.Spectrum.from_file(inFile)
outFile = r""
"""
Instantaneous size change some time ago.

params = (nu,T)
ns = (n1,)

nu: Ratio of contemporary to ancient population size
T: Time in the past at which size change happened (in units of 2*Na 
   generations) 
n1: Number of samples in resulting Spectrum
pts: Number of grid points to use in integration.
"""
lb = [0.01, 0.001]
ub = [10, 5]
p0 = [0.05, 0.1]

for i in range(1, 101):
    p0 = [0.05, 0.1]
    p0 = dadi.Misc.perturb_params(p0, fold=2, upper_bound=ub, lower_bound=lb)

    func = dadi.Demographics1D.two_epoch
    # Make the extrapolating version of our demographic model function.
    func_ex = dadi.Numerics.make_extrap_log_func(func)

    popt = dadi.Inference.optimize(p0, fs, func_ex, pts_l, lower_bound=lb, upper_bound=ub, verbose=False, full_output=True, maxiter=300)

    #Calculate the best-fit model AFS.
    model = func_ex(popt[0], ns, pts_l)
    theta = dadi.Inference.optimal_sfs_scaling(model, fs)
    # Likelihood of the data given the model AFS.
    ll_model = dadi.Inference.ll_multinom(model, fs)
    Nanc = theta/(2.5e-8*1000000*4)
    Ncurr = popt[0][0]*Nanc
    T = popt[0][1]*(2*Nanc)
    with open(outFile, 'a') as f:
        f.write(str(popt[0][0]) + '\t' + str(popt[0][1]) + '\t' + 
                str(theta) + '\t' + str(Nanc) + '\t' + str(Ncurr) + '\t' + str(T) + "\t" + str(ll_model) + '\n')
    if(i%20==0):
        print("rep " + str(i) + " complete")
